<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/1_Twitter_%E7%89%B9%E5%AE%9A%E3%81%AE%E6%96%87%E7%AB%A0%E3%82%92%E6%8A%95%E7%A8%BF%E3%81%97%E3%81%9F%E3%83%A6%E3%83%BC%E3%82%B6%E3%83%BC%E3%82%92%E9%9B%86%E3%82%81%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>Twitter 特定の文章を投稿したユーザーを集める</center>

##<center>特定の文章 = MBTI診断の結果</center>

#0. 作業環境の準備

##0.1. Google Driveをマウント

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

##0.2. カレントディレクトリへ移動

In [7]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在日時の取得

In [ ]:
import datetime
from pytz import timezone

def Pull_Datetime():
    dates=datetime.datetime.now(timezone('Asia/Tokyo'))
    exept_microsec=dates.strftime("%Y-%m-%d-%H-%M")
    print('[Today : %s]' % str(exept_microsec))

    DATETIME=exept_microsec
    return DATETIME

DATETIME=Pull_Datetime()

[Today : 2021-08-24-00-36]


##0.6. ファイル名に含む単語 (任意) を設定する。

ファイルの中身を明示する内容がオススメ。

In [ ]:
KEYWORDS_FileName= '2017'

##0.5. 作業ディレクトリの作成

In [ ]:
import os
OUTPUT_PATH =  './Assets/Assets_Output/'
OUTPUT_DIR=OUTPUT_PATH+ "PersonalEstim_%s/" % DATETIME
OUTPUT_DIR_RESULTS=OUTPUT_PATH+ "PersonalEstim_%s/results/" % DATETIME

KEYWORDS_TypeMBTI = '2017-2018'

try:
  os.mkdir(OUTPUT_DIR)
except:
  print(OUTPUT_DIR, '    already exists')
  print()
try:
  os.mkdir(OUTPUT_DIR_RESULTS)
except:
  print(OUTPUT_DIR_RESULTS, '    the directory already exists')
  print()

#3. Twitter APIを利用して、特定の投稿をしたユーザー情報を取得する。

特定の投稿 = MBTI診断の結果

##3.1. 「Twiiter APIのエンドポイントを作成」する関数

今回のTwitter API のURLは「特定の投稿をしたユーザー情報を取得」するエンドポイントである。

In [ ]:
import yaml
import os # For saving access tokens and for file management when creating and adding to the dataset
import requests # For sending GET requests from the API
import Twitter_API_Key

def Create_url_SearchTweets(keyword, start_date, end_date, max_results = 10):
    #Change to the endpoint you want to collect data from
    search_url = "https://api.twitter.com/2/tweets/search/all" 
    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
                    
    return (search_url, query_params)

##3.2. 「エンドポイントを利用してTwitter APIへアクセス」する関数

In [ ]:
def Connect_to_endpoint(url, params, next_token = None):
    #prepare params
    params['next_token'] = next_token   #params object received from create_url function
    #pull bearer token
    bearer_token = Twitter_API_Key.Auth_FullAchieve()
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    #request Twtitter
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()

##3.3. 「特定の投稿をしたユーザー情報を取得」する関数

In [ ]:
import dateutil.parser
import pandas as pd

def Search_Tweets(keyword, start_time, end_time, max_results, next_token, KEYWORDS_FileName, count_index):
  url = Create_url_SearchTweets(keyword, start_time, end_time, max_results)
  print(url)
  json_SearchTweets= Connect_to_endpoint(url[0], url[1], next_token)
  # json_SearchTweets
  print(json_SearchTweets)
  list_EachData=json_SearchTweets['data']

  list_csv=[]
  list_csv.clear()
  list_header=['author_id', 'text', 'tweet_id', 'tweet_time', 'lang']
  for element in list_EachData:
    #print(element['text'])
    tweet_AuthorID=element['author_id']
    tweet_Texts=element['text']
    tweet_id = element['id']
    tweet_time = dateutil.parser.parse(element['created_at'])
    tweet_lang = element['lang']

    list_sub_row=[tweet_AuthorID, tweet_Texts, tweet_id, tweet_time, tweet_lang]
    list_csv.append(list_sub_row)

  df_SearchTweets=pd.DataFrame(list_csv, columns=list_header)
  df_SearchTweets.to_csv('%sUserList_%s.csv' % (OUTPUT_DIR_RESULTS, count_index))
  print('%sUserList_%s.csvを作成しました。'% (OUTPUT_DIR_RESULTS, count_index))

  return df_SearchTweets, tweet_time

##3.5 これまでに作成した関数を動かす。

In [ ]:
import time

#Inputs for the request
keyword = "#16Personalities 16personalities.com lang:ja"
max_results = 500
start_time = "2017-01-01T00:00:00.000Z"
end_time='2017-12-31T23:59:59.000Z'
# start_time = "2018-12-31T00:00:00.000Z"
# end_time='2018-12-31T23:59:59.000Z'
df_2017to2018=None

for count_index in range(0, 10000):
  print(count_index)
  next_token = None
  df_SearchTweets, end_time=Search_Tweets(keyword, start_time, end_time, max_results, next_token, KEYWORDS_TypeMBTI, count_index)

  b=str(end_time).replace(' ', 'T')
  #print(b.split('+'))
  c=b.split('+')
  end_time=c[0] + '.000Z'
  #print(d)
  
  df_2017to2018=pd.concat([df_2017to2018, df_SearchTweets], axis=0)
  # except:
  #   print('passしました')
  #   pass

  #time.sleep(5)
  print(end_time)
  
df_2017to2018.to_csv('%sUserListFinal_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))
#df_SearchTweets.to_csv('%sUserList_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))

##3.6. 作成したファイルの表示

個人の特定に繫がる情報は伏せてます。

In [13]:
import pandas as pd
#df_2017to2018=pd.read_csv('./Assets/Assets_Output/PersonalEstim_2021-08-24-00-36/results/UserListFinal_2017.csv', engine='python')

#df_SearchTweets[['text', 'tweet_time', 'lang']].head()

df_2017to2018[['text', 'tweet_time', 'lang']].head()

,text,tweet_time,lang
0,私の性格タイプは “建築家” です。あなたのタイプは何ですか？ https://t.co/c...,2017-12-31 15:43:25+00:00,ja
1,私の性格タイプは “仲介者” です。あなたのタイプは何ですか？ https://t.co/y...,2017-12-31 14:57:52+00:00,ja
2,私の性格タイプは “広報運動家” です。あなたのタイプは何ですか？ https://t.co...,2017-12-31 14:51:55+00:00,ja
3,仲介者型の性格 (INFP-A / INFP-T) https://t.co/HtTGyon...,2017-12-31 13:56:54+00:00,ja
4,私の性格タイプは “仲介者” です。あなたのタイプは何ですか？ https://t.co/t...,2017-12-31 13:41:12+00:00,ja


In [ ]:
#df_2017to2018.to_csv('%sUserListFinal_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))

#Z. デバック用

In [ ]:
print('%sUserListFinal_%s.csv' % (OUTPUT_DIR_RESULTS, KEYWORDS_FileName))
list_AuthorID=df_2017to2018['author_id'].values.tolist()

list_AuthorID.sort()
print(list_AuthorID)

with open('/content/drive/MyDrive/sample.txt', 'w') as f:
  for i in range(len(list_AuthorID)):
    print(list_AuthorID[i], file=f)


with open('/content/drive/MyDrive/sample.txt') as f:
  list_AuthorID=f.readlines()
  for i in range(len(list_AuthorID)):
    list_AuthorID[i]=list_AuthorID[i].rstrip()
  print(list_AuthorID)